# Risk Indicator and Assessment Reporting

Here is an demonstration organization-centric application that aims to provide qulatative reports addresing the risks on the customer database. The Risk Indicator and Assessment Reporting application’s primary objective is to streamline and enhance the risk management process for an BFSI organizations. At its core, the application leverages the advanced AI capabilities of OpenAI's GPT-3.5 Turbo. GPT-3.5 Turbo possesses remarkable natural language processing abilities, which enable it to understand and interpret complex financial data and risk-related information.

By automating risk assessment, providing comprehensive reports, offering real-time insights, and promoting better decision-making and compliance, the application equips organizations with the tools they need to navigate complex financial landscapes and mitigate potential risks effectively.


## Setting up

In this demonstration we will would require `openai` module.

In [ ]:
!pip install openai

## Connecting the Drive

This is an optional part. Here we will connect our google drive to the notebook environment, this way we could access file directory directy from our drive in this notebook.

In [ ]:
from google.colab import drive
drive.mount("/content/gDrive")

Mounted at /content/gDrive


## Importing the dependencies and preprequisities

In [ ]:
import pandas as pd
import numpy as np
import openai

* In here we are importing `pandas` module to operate on the dataframe. We will use this module for generating data sample, data preprocessing and data cleaning.

* Then we would require `numpy` module for generating numerical value and operate on them.

* Since we would be using the GPT-3.5 turbo in the backend to analyze the data and generate texts. Therefore we would require `openai` module to interact with the model. It would enables us to make API calls to use the capabilities of OpenAI's language models, such as generating text, completing prompts, answering questions, and more.

## Generating sample dataset

**ClientID**
* A unique identifier for each customer in the dataset.
* It allows easy referencing and tracking of individual customers, making it a crucial primary key for the dataset.

**LoanAmount**:
* Represents the amount of money requested by each customer as a loan.
* This information is essential for analyzing loan sizes, assessing risk associated with higher loan amounts, and studying trends in customer borrowing behavior.

**LoanTerm**:
* Indicates the duration of each loan in months.
* It provides insights into the length of customer commitments and allows analysis of loan terms' impact on default rates, customer satisfaction, and profitability.

**RepaymentDate**:
* Specifies the date by which the customer is expected to repay the loan.
* This information is vital for tracking loan repayment timelines and assessing the accuracy of repayments.

**EarlyRepayment**:
* A binary indicator (0 or 1) that denotes whether a customer made an early repayment (1) or not (0).
* This data helps in understanding early repayment behavior, which may be influenced by various factors such as financial stability or customer satisfaction.

**Default**:
* A binary indicator (0 or 1) that represents whether a customer defaulted on the loan (1) or not (0).
* It is a critical target variable for building models to predict loan defaults and assess loan risk.

**CreditScore**:
* Provides the credit score of each customer.
* Credit scores reflect the creditworthiness of customers and are instrumental in evaluating their ability to repay loans.

**MonthlyIncome**:
* Contains the monthly income of customers.
* This information is essential for assessing a customer's ability to repay loans and determining suitable loan amounts.

**Age**:
* Represents the age of each customer.
* Age is a demographic variable that can influence loan preferences, risk profiles, and financial stability.

**EmploymentDuration**:
* Indicates the duration of the customer's current employment in years.
* It can provide insights into job stability and how it affects loan repayment capabilities.

**NumOfPreviousLoans**:
* Records the number of previous loans taken by each customer.
* This information is valuable for understanding customer borrowing history and its impact on future loan behaviors.

**InterestRate**:
* Contains the interest rate charged on each loan.
* This data helps in analyzing the relationship between interest rates and loan performance, as well as understanding how interest rates impact customer decisions.

**CashFlowVariability**:
* Represents a measure of the customer's cash flow variability.
* It provides insights into the stability of a customer's income and can be used to assess their capacity to handle loan repayments.

In [ ]:
data = {
    'ClientID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
    'LoanAmount': [10000, 5000, 15000, 8000, 12000, 6000, 25000, 3000, 18000, 7500, 14000, 20000, 9000, 6500, 30000, 8000, 6000, 4000, 20000, 10000],
    'LoanTerm': [12, 6, 24, 18, 12, 9, 36, 6, 12, 24, 18, 36, 12, 9, 48, 12, 9, 6, 36, 18],
    'RepaymentDate': ['2023-05-15', '2023-08-20', '2023-12-10', '2023-11-05', '2023-06-30', '2023-09-25',
                      '2023-07-10', '2023-12-05', '2023-06-20', '2023-10-15', '2023-09-30', '2023-07-05',
                      '2023-08-10', '2023-11-30', '2023-05-20', '2023-07-25', '2023-09-20', '2023-08-05',
                      '2023-07-20', '2023-10-30'],
    'EarlyRepayment': [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1],
    'Default': [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1],
    'CreditScore': [720, 640, 800, 580, 700, 750, 690, 620, 820, 550, 680, 750, 600, 700, 780, 640, 680, 710, 760, 690],
    'MonthlyIncome': [5000, 3000, 8000, 2000, 4500, 6000, 4000, 3500, 9000, 1800, 4200, 6500, 2500, 3200, 10000, 3800, 2800, 3500, 8000, 4500],
    'Age': [35, 28, 45, 22, 33, 40, 28, 30, 50, 21, 38, 48, 25, 35, 42, 26, 30, 32, 45, 40],
    'EmploymentDuration': [5, 2, 10, 1, 7, 8, 3, 1, 15, 0, 6, 12, 2, 4, 18, 1, 2, 3, 10, 8],
    'NumOfPreviousLoans': [2, 1, 3, 0, 2, 4, 1, 0, 5, 0, 2, 3, 1, 2, 6, 0, 1, 1, 4, 3],
    'InterestRate': [0.09, 0.07, 0.1, 0.08, 0.09, 0.06,0.12, 0.1, 0.08, 0.09,0.09, 0.07, 0.06,0.12, 0.1, 0.08, 0.09, 0.06, 0.12, 0.1],
    'CashFlowVariability': [0.15, 0.2, 0.18, 0.3, 0.12, 0.18, 0.25, 0.15, 0.2, 0.18, 0.3, 0.12, 0.18, 0.25, 0.15, 0.2, 0.18, 0.3, 0.12, 0.18]
}

## Data Operations

### Data Loading

We are converting the above data dictionary to pandas dataframe to operate on it.

In [ ]:
df = pd.DataFrame(data)
df[:2]

,ClientID,LoanAmount,LoanTerm,RepaymentDate,EarlyRepayment,Default,CreditScore,MonthlyIncome,Age,EmploymentDuration,NumOfPreviousLoans,InterestRate,CashFlowVariability
0,1,10000,12,2023-05-15,0,0,720,5000,35,5,2,0.09,0.15
1,2,5000,6,2023-08-20,1,0,640,3000,28,2,1,0.07,0.20


### Feature Enginnering

In here are converting the `RepaymentDate` column to datetime format. We have perform this operation so as to create a new column which would reflect the days remaining to complete the loan.

In [ ]:
# Convert 'RepaymentDate' to datetime
df['RepaymentDate'] = pd.to_datetime(df['RepaymentDate'])

# Calculate the time remaining for loan repayment
df['TimeRemaining'] = (df['RepaymentDate'] - pd.Timestamp('today')).dt.days

* The 'RepaymentDate' column contains the dates in a string format like 'YYYY-MM-DD'.The `pd.to_datetime()` function is a pandas method used to convert a sequence of strings representing dates to pandas datetime objects. After this operation, the 'RepaymentDate' column will be a pandas datetime data type, allowing easy manipulation of dates.
* The `df['RepaymentDate'] - pd.Timestamp('today')` calculates the time difference between the 'RepaymentDate' and the current date (pd.Timestamp('today')), which gives a pandas Timedelta object.
* The .dt.days attribute of the Timedelta object extracts the time difference in days and creates a new numeric Series representing the time remaining for loan repayment in days.


This information helps ensure timely follow-ups and reminders to customers to avoid late payments or defaults and also early payments which is also an issue.

In here we are adding a new column `LiquidityRisk` in the dataframe. Liquidity risk refers to the risk faced by financial institutions, such as lenders, when they do not have sufficient cash or liquid assets to meet their financial obligations, including funding customer withdrawals or honoring loan repayments. Calculating liquidity risk in the context of loans is essential for several reasons:

* To effectively manage their cash flow and ensure they have enough funds to meet their obligations.
* To identify loans that may pose a higher risk of draining their cash reserves.
* Lenders can evaluate the adequacy of their cash reserves. If the liquidity risk exceeds an acceptable threshold, lenders may consider increasing their cash reserves or adjusting their lending practices.

In [ ]:
lender_cash_reserves = 1000000  # Replace this value with actual cash reserve information

# Calculate liquidity risk based on loan size, proportion of early repayments, and cash reserves
df['LiquidityRisk'] = df['LoanAmount'] * df['EarlyRepayment'] / lender_cash_reserves

IN the above code we are caculating the liquidity risk for each loan in the DataFrame and stores the result in a new column named 'LiquidityRisk'.
The liquidity risk is determined by multiplying the loan amount (df['LoanAmount']) by the proportion of early repayments (df['EarlyRepayment']) and then dividing the result by the lender's cash reserves (lender_cash_reserves).

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ClientID             20 non-null     int64         
 1   LoanAmount           20 non-null     int64         
 2   LoanTerm             20 non-null     int64         
 3   RepaymentDate        20 non-null     datetime64[ns]
 4   EarlyRepayment       20 non-null     int64         
 5   Default              20 non-null     int64         
 6   CreditScore          20 non-null     int64         
 7   MonthlyIncome        20 non-null     int64         
 8   Age                  20 non-null     int64         
 9   EmploymentDuration   20 non-null     int64         
 10  NumOfPreviousLoans   20 non-null     int64         
 11  InterestRate         20 non-null     float64       
 12  CashFlowVariability  20 non-null     float64       
 13  TimeRemaining        20 non-null     

In here we are adding two new columns to the dataframe `FullTermInterestIncome` and `InterestIncomeDifference` which represent the interest recived by the lender and the difference respectively.

In [ ]:
full_term_interest_income = [0.08, 0.06, 0.09, 0.07, 0.08, 0.05, 0.07, 0.06, 0.09, 0.1, 0.08, 0.07, 0.09, 0.06, 0.08, 0.07, 0.09, 0.06, 0.05, 0.09]
df['FullTermInterestIncome'] = df['LoanAmount'] * full_term_interest_income * df['LoanTerm'] / 12
df['InterestIncomeDifference'] = df['EarlyRepayment'] * df['LoanAmount'] * (df['InterestRate'] * df['TimeRemaining'] / 365 - full_term_interest_income)


* The variable `full_term_interest_income` is a list of interest rates corresponding to each loan in the DataFrame. These rates represent the interest income a lender would receive for each loan if it is held for the full loan term (in years).
* The variable `FullTermInterestIncome` calculates the full-term interest income for each loan in the DataFrame and stores the result in a new column named 'FullTermInterestIncome'. It multiplies the 'LoanAmount' by the corresponding 'full_term_interest_income' value and the loan term ('LoanTerm'), then divides the result by 12 to convert the loan term from months to years.
* The variable `InterestIncomeDifference` calculates the difference in interest income for early repayments compared to full-term repayments and stores the result in a new column named 'InterestIncomeDifference'.
It multiplies the 'EarlyRepayment' binary indicator (1 for early repayment, 0 otherwise) by the 'LoanAmount' and then computes the difference between the interest income for the remaining days ('TimeRemaining') based on the actual 'InterestRate' and the full-term interest income ('full_term_interest_income').

## Final dataframe

In [ ]:
df[:2]

,ClientID,LoanAmount,LoanTerm,RepaymentDate,EarlyRepayment,Default,CreditScore,MonthlyIncome,Age,EmploymentDuration,NumOfPreviousLoans,InterestRate,CashFlowVariability,TimeRemaining,LiquidityRisk,FullTermInterestIncome,InterestIncomeDifference
0,1,10000,12,2023-05-15,0,0,720,5000,35,5,2,0.09,0.15,-74,0.000,800.0,-0.000000
1,2,5000,6,2023-08-20,1,0,640,3000,28,2,1,0.07,0.20,23,0.005,150.0,-277.945205


In [ ]:
converted_data = df.to_dict(orient='list')

In [ ]:
data_fed = {
    'ClientID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
    'LoanAmount': [10000, 5000, 15000, 8000, 12000, 6000, 25000, 3000, 18000, 7500, 14000, 20000, 9000, 6500, 30000, 8000, 6000, 4000, 20000, 10000],
    'LoanTerm': [12, 6, 24, 18, 12, 9, 36, 6, 12, 24, 18, 36, 12, 9, 48, 12, 9, 6, 36, 18],
    'RepaymentDate': ['2023-05-15', '2023-08-20', '2023-12-10', '2023-11-05', '2023-06-30', '2023-09-25',
                      '2023-07-10', '2023-12-05', '2023-06-20', '2023-10-15', '2023-09-30', '2023-07-05',
                      '2023-08-10', '2023-11-30', '2023-05-20', '2023-07-25', '2023-09-20', '2023-08-05',
                      '2023-07-20', '2023-10-30'],
    'EarlyRepayment': [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1],
    'Default': [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1],
    'CreditScore': [720, 640, 800, 580, 700, 750, 690, 620, 820, 550, 680, 750, 600, 700, 780, 640, 680, 710, 760, 690],
    'MonthlyIncome': [5000, 3000, 8000, 2000, 4500, 6000, 4000, 3500, 9000, 1800, 4200, 6500, 2500, 3200, 10000, 3800, 2800, 3500, 8000, 4500],
    'Age': [35, 28, 45, 22, 33, 40, 28, 30, 50, 21, 38, 48, 25, 35, 42, 26, 30, 32, 45, 40],
    'EmploymentDuration': [5, 2, 10, 1, 7, 8, 3, 1, 15, 0, 6, 12, 2, 4, 18, 1, 2, 3, 10, 8],
    'NumOfPreviousLoans': [2, 1, 3, 0, 2, 4, 1, 0, 5, 0, 2, 3, 1, 2, 6, 0, 1, 1, 4, 3],
    'InterestRate': [0.09, 0.07, 0.10, 0.08, 0.09, 0.06, 0.12, 0.10, 0.08, 0.09, 0.09, 0.07, 0.06, 0.12, 0.10, 0.08, 0.09, 0.06, 0.12, 0.10],
    'CashFlowVariability': [0.15, 0.20, 0.18, 0.30, 0.12, 0.18, 0.25, 0.15, 0.20, 0.18, 0.30, 0.12, 0.18, 0.25, 0.15, 0.20, 0.18, 0.30, 0.12, 0.18],
    'TimeRemaining': [-73, 24, 136, 101, -27, 60, -17, 131, -37, 80, 65, -22, 14, 126, -68, -2, 55, 9, -7, 95],
    'LiquidityRisk': [0.000, 0.005, 0.000, 0.000, 0.012, 0.000, 0.000, 0.003, 0.000, 0.000, 0.014, 0.000, 0.009, 0.000, 0.030, 0.000, 0.000, 0.000, 0.000, 0.010],
    'FullTermInterestIncome': [800.0, 150.0, 2700.0, 840.0, 960.0, 225.0, 5250.0, 90.0, 1620.0, 1500.0, 1680.0, 4200.0, 810.0, 292.5, 9600.0, 560.0, 405.0, 120.0, 3000.0, 1350.0],
    'InterestIncomeDifference': [-0.000000, -276.986301, -0.000000, -0.000000, -1039.890411, -0.000000, -0.000000, -72.328767, -0.000000, -0.000000, -895.616438, -0.000000, -789.287671, -0.000000, -2958.904110, -0.000000, -0.000000, -0.000000, -0.000000, -639.726027]
}

we will calculate:

* Early Repayment Statistics
* Opportunity Cost
* Interest Income Reduction
* Cash Flow Variability
* Loss of Profit
* Liquidity Risk

In [ ]:
openai.api_key = 'your-api-key'

In [ ]:
completion_1 = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  temperature = 0.4,
  messages=[
    {"role": "user",
     "content": f"""
     Below is my data could you calculate the following indicators and provide me an report  based on the data provided and do not include any code.
      1. Early Repayment Statistics
      2. Opportunity Cost
      3. Interest Income Reduction
      4. Cash Flow Variability
      5. Loss of Profit
      6. Liquidity Risk
     Data:
     {data_fed}
     """
     }]
)

In [ ]:
response_1=completion_1.choices[0].message.content

In [ ]:
response_1

'Report:\n\n1. Early Repayment Statistics:\n   - Number of loans with early repayment: 8\n   - Percentage of loans with early repayment: 40%\n\n2. Opportunity Cost:\n   - Total interest income from full term: $27,267.50\n   - Total interest income from actual repayments: $22,308.56\n   - Opportunity cost: $4,958.94\n\n3. Interest Income Reduction:\n   - Total interest income reduction due to early repayment: $5,958.90\n   - Average interest income reduction per loan: $744.86\n\n4. Cash Flow Variability:\n   - Average cash flow variability: 0.186\n\n5. Loss of Profit:\n   - Number of defaulted loans: 4\n   - Percentage of defaulted loans: 20%\n   - Total loss of profit from defaulted loans: $5,400.00\n   - Average loss of profit per defaulted loan: $1,350.00\n\n6. Liquidity Risk:\n   - Average liquidity risk: 0.007\n\nNote: The above calculations are based on the provided data and do not include any external factors or assumptions.'

In [ ]:
def export_string_to_txt_file(file_path, content):
    try:
        # Open the file in write mode ('w')
        with open(file_path, 'w') as file:
            # Write the content to the file
            file.write(content)
        print(f"String successfully exported to '{file_path}'.")
    except IOError:
        print(f"Error: Unable to write to the file '{file_path}'.")

In [ ]:
file_path = "/content/gDrive/MyDrive/29th July workshop/risk_indicator_report.txt"
export_string_to_txt_file(file_path, response_1)

String successfully exported to '/content/gDrive/MyDrive/29th July workshop/risk_indicator_report.txt'.
